In [1]:
import torch
import torchvision.models as models

model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))
model.eval()


from PIL import Image
from torchvision import transforms
from sklearn.metrics.pairwise import cosine_similarity

# image_path = 'data/DAM/010M03A1116X9000.jpeg'
# input_image = Image.open(image_path)

image_paths = [
	'data/DAM/012B03A3985X5902.jpeg',
	'data/DAM/012A09A3232X5597.jpeg',
	'data/DAM/14DDN978A133C568.jpeg'
]


preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Extract features for each image
features_list = []
for image_path in image_paths:
    input_image = Image.open(image_path)
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        features = model(input_batch)

    # Convert features to a numpy array and flatten it
    features_list.append(features.cpu().numpy().flatten())

# Compute cosine similarity between features
cosine_similarities = cosine_similarity(features_list)

# Print the cosine similarity matrix
print("Cosine Similarity Matrix:")
print(cosine_similarities)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Cosine Similarity Matrix:
[[1.         0.7414424  0.6193912 ]
 [0.7414424  1.         0.67094374]
 [0.6193912  0.67094374 0.9999997 ]]
